In [20]:
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np

mnist = fetch_openml('mnist_784', version=1)

X, y = mnist['data'], mnist['target']
X_train, X_test, y_train, y_test = np.array(X)[:60000], np.array(X)[60000:], np.array(y)[:60000], np.array(y)[60000:]

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


# 1.

In [16]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=6)
X_train_small, y_train_small = None, None

for train_index, test_index in skf.split(X_train, y_train):
    X_train_small, y_train_small = X_train[train_index], y_train[train_index]
    break

In [17]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

knn_clf = KNeighborsClassifier()
pipeline = Pipeline([
    ('std_scaler', StandardScaler()),
    ('knn', knn_clf)
])
param_grid = [{
    'knn__weights': ['uniform', 'distance'],
    'knn__n_neighbors': [3, 4, 5, 6],
    'knn__n_jobs': [-1]
}]

grid_search = GridSearchCV(pipeline, param_grid, cv=3,
                           scoring='accuracy',
                           return_train_score=True)
grid_search.fit(X_train_small, y_train_small)
print(grid_search.best_score_)
print(grid_search.best_params_)

0.940660003552599


In [22]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score


knn_clf = KNeighborsClassifier(n_neighbors=4, weights='distance', n_jobs=-1)
pipeline = Pipeline([
    # ('std_scaler', StandardScaler()),  # decreases accuracy by 2%
    ('knn', knn_clf)
])
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.9714


# 2.

In [29]:
from scipy.ndimage import shift


def shift_img(img, dy, dx):
    reshaped_img = img.reshape((28, 28))
    reshaped_img = shift(reshaped_img, [dy, dx])
    return reshaped_img.reshape([-1])


X_train_left_shift = np.array([shift_img(img, 0, -5) for img in X_train])
X_train_right_shift = np.array([shift_img(img, 0, 5) for img in X_train])
X_train_up_shift = np.array([shift_img(img, -5, 0) for img in X_train])
X_train_down_shift = np.array([shift_img(img, 5, 0) for img in X_train])

X_train_expanded = np.concatenate((X_train, X_train_left_shift,
                                   X_train_right_shift, X_train_up_shift,
                                   X_train_down_shift), dtype=X_train.dtype)
y_train_expanded = np.tile(y_train, 5)

knn_clf.fit(X_train_expanded, y_train_expanded)
y_pred = knn_clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.9708
